In [1]:
import numpy as np
from scipy import linalg
from scipy.optimize import minimize

In [ ]:
#                 (0,244)--------(294,244)
#                     |               |
# Polygone ->         |               |
#                     |               |
#                   (0,0)--------(294, 0)

In [9]:
pol1 = (582/1280, 271/720)
pol1_2 = (486/1280, 384/720)
pol2 = (371/1280, 520/720)
pol2_2 = (614/1280, 601/720)
pol3 = (945/1280, 710/720)
pol3_2 = (976/1280, 471/720)
pol4 = (994/1280, 317/720)
pol4_2 = (764/1280, 290/720)


x_res, y_res = (1920, 1080)# resolution
polygon = np.array([(pol1[0] * x_res, pol1[1] * y_res), 
                    (pol1_2[0] * x_res, pol1_2[1] * y_res), 
                    (pol2[0] * x_res, pol2[1] * y_res),
                    (pol2_2[0] * x_res, pol2_2[1] * y_res),
                    (pol3[0] * x_res, pol3[1] * y_res),
                    (pol3_2[0] * x_res, pol3_2[1] * y_res),
                    (pol4[0] * x_res, pol4[1] * y_res), 
                    (pol4_2[0] * x_res, pol4_2[1] * y_res)], dtype=int)#points for polygone in the center


def solveForCoefs(params):
    """
    REQUIRE:
    pointsW = [[xw,yw],[xw2,yw2],...,[xwN,ywN]] - WORLD COORDS
    pointsPX = [[u,v],[u2,v2],...,[uN,vN]] - IMAGE COORDS
    """
    fx, fy, cx, cy, phi1, phi2, phi3, tx, ty, tz = params

    pointsW = [(0, 244), (0,122), (0, 0), (147, 0), (294, 0), (294, 122), (294, 244), (147, 244)]
    pointsPX = polygon

    # внутр. матрица
    M_int = np.array([[fx, 0, cx, 0],
                     [0, fy, cy, 0],
                     [0, 0, 1, 0]])
    # 3 матрицы поворота
    m_rot1 = np.array([[1, 0, 0], [0, np.cos(phi1), -np.sin(phi1)], [0, np.sin(phi1), np.cos(phi1)]])
    m_rot2 = np.array([[np.cos(phi2), 0, -np.sin(phi2)], [0, 1, 0], [np.sin(phi2), 0, np.cos(phi2)]])
    m_rot3 = np.array([[np.cos(phi3), -np.sin(phi3), 0], [np.sin(phi3), np.cos(phi3), 0], [0, 0, 1]])
    m_rot = np.matmul(np.matmul(m_rot1, m_rot2), m_rot3) # произведение матриц поворота

    # расширение матрицы поворота (справа - вектором смещения, снизу стркой [0,0,0,1]). Теперь это внешняя матрица
    M_ext = np.vstack((np.hstack((m_rot, [[tx], [ty], [tz]])), [0, 0, 0, 1]))
    # произведение внутренней и вшешних матриц
    A = np.matmul(M_int, M_ext)

    # Задаем метрику кврадартичного отклонения
    metric = 0
    # Проходимся по всем world и внутренним точкам. Считаем сумму квадратичных отклонений
    for W, PX in zip(pointsW, pointsPX):
        X = np.append(W, [0, 1])
        y_pred = np.matmul(A, X)
        y_pred = y_pred / y_pred[-1]
        y_pred += [1920/2, 1080/2, 0]
        metric += np.sum((PX - y_pred[:2])**2)

    metric = metric / len(pointsW)
    print(metric)
    return metric


# x0 = [0.5, 0.5, 0.5, 0.5, 5, 5, 5, 243, -120, 200]
x0 = [187.08826537858658,-256.73158416715137,49.6722205130153,980.7968744515073,75.53078665035575,31.395759410121656,62.44511493820521,-166.01045337973667,196.03770232314747,68.71317060268947]
out = minimize(solveForCoefs, x0, method='SLSQP')
print(*out.x, sep=",")

52.88674635983141
52.88674635983748
52.88674635985332
52.88674635982916
52.886746359836614
52.88674636573509
52.886746371796725
52.88674635642662
52.886746359822745
52.886746359815916
52.88674635988964
7378055.163169945
210529.69080382425
941.685010206413
61.209093865297696
52.96937181588195
52.88756558341601
52.88675390143836
52.88674637023698
52.886746344053684
187.08826537856842,-256.7315841672169,49.67222051302203,980.7968744514917,75.5307866326979,31.3957593743335,62.44511494838892,-166.01045337971075,196.03770232319383,68.7131706025153


In [10]:
def getRealcoords(pt):
    fx, fy, cx, cy, phi1, phi2, phi3, tx, ty, tz = \
        [187.08826537856842,-256.7315841672169,49.67222051302203,980.7968744514917,75.5307866326979,31.3957593743335,62.44511494838892,-166.01045337971075,196.03770232319383,68.7131706025153]

    
    M_int = np.array([  [fx, 0, cx, 0],
                        [0, fy, cy, 0],
                        [0, 0, 1, 0]])

    m_rot1 = np.array([[1, 0, 0], [0, np.cos(phi1), -np.sin(phi1)], [0, np.sin(phi1), np.cos(phi1)]])
    m_rot2 = np.array([[np.cos(phi2), 0, -np.sin(phi2)], [0, 1, 0], [np.sin(phi2), 0, np.cos(phi2)]])    
    m_rot3 = np.array([[np.cos(phi3), -np.sin(phi3), 0], [np.sin(phi3), np.cos(phi3), 0], [0, 0, 1]])
    m_rot = np.matmul(np.matmul(m_rot2, m_rot1), m_rot3)
    M_ext = np.vstack((np.hstack((m_rot, [[tx], [ty], [tz]])), [0, 0, 0, 1]))
    A = np.matmul(M_int, M_ext)

    b = np.array([pt[0], pt[1], 0, 1])

    answ = np.matmul(A, b)
    answ = answ / answ[-1]
    answ += [1920/2, 1080/2, 0]
    return answ[:2]




print(polygon)
print(getRealcoords([0, 0]))

[[ 873  406]
 [ 729  576]
 [ 556  780]
 [ 921  901]
 [1417 1065]
 [1464  706]
 [1491  475]
 [1146  435]]
[557.6685146  788.34530174]
